## 通过调用微博API的方法将微博评论写入数据库

In [1]:
from weibo import APIClient 
import webbrowser
import pymysql,re,time

In [ ]:
# import importlib,sys
# importlib.reload(sys)

In [2]:
import sys
sys.getfilesystemencoding()

'utf-8'

In [3]:
APP_KEY = '150779162' #获取的App Key 
APP_SECRET = 'f9e4856afaee46b34c6c23874372a3f7' #获取的AppSecret 
CALLBACK_URL = 'https://api.weibo.com/oauth2/default.html' #回调链接 

In [ ]:
client = APIClient(app_key=APP_KEY, app_secret=APP_SECRET, redirect_uri=CALLBACK_URL) 
url = client.get_authorize_url() 
webbrowser.open_new(url) #打开默认浏览器获取code参数 

In [9]:
code = input("please input code : ")
result = client.request_access_token(code)
print(result)
access_token = result.access_token
expires_in = result.expires_in
client.set_access_token(access_token, expires_in)

please input code :  810195e1f7ed19ecc9639977b4538ea7


{'access_token': '2.007Ni7YF0yXeMK98ee6af6e4TCDV6C', 'expires': 1784525473, 'expires_in': 1784525473, 'uid': '5089426080'}


In [44]:
# r = client.comments.show.get(id = 4660583661568436, count = 200, page = 1)
# obj = r.comments

In [ ]:
# r.comments

In [45]:
# type(obj)

list

In [ ]:
# for i in obj:
#     print(i)  # 回复@你奈我何-Kris-:ex
#     print(i.user.screen_name)

In [ ]:
for i in range(1,10):
    res = client.comments.show.get(id = 4660583661568436, count = 200, page = i)
    if len(res.comments):
        print('第 %s 页' % i)
        
# ---------获取评论内容-----------
        for co in r.comments:
            created_at = co.created_at  # 时间
            comment_id = co.id   # 评论id
            comment_obj = co.text  # 评论内容提取
            text = re.sub('回复.*?:','',comment_obj) 
            user_name = co.user.screen_name
            followers = co.user.followers_count
            follow = co.user.friends_count
            province = co.user.province
            
            print('评论者：%s,粉丝数：%s,关注数：%s,所在省份编号：%s' % (user_name,followers,follow,province))
            print('\n')
                  
# ----------连接数据库-----------
            db =pymysql.connect(host='localhost',user='root',password='',database = 'pachong', charset="utf8",use_unicode = False)
            cur = db.cursor()
            sql = "INSERT INTO wuyifan(created_at, comment_id, text, user_name, followers, follow, province) values(%s,%s,%s,%s,%s,%s,%s)"
            columns = (created_at, comment_id, text, user_name, followers, follow, province)
            try:
                cur.execute(sql,columns)
                db.commit()
                cur.close()
                db.close()
            except Exception as e:
                print(e)
                db.rollback()
                
            comment_num += 1
        time.sleep(4)
    else:
        break